# Robust Optimization for Genetic Selection

This notebook provides a didactic for using Python to solve the multi-objective optimization problem which arises in the context of robust genetic selection. It has been tested with Python 3.10 specifically and there are some standard packages which this depends on, imported below.

In [1]:
import numpy as np                  # defines matrix structures
from qpsolvers import solve_qp      # used for quadratic optimization
import gurobipy as gp               # Gurobi optimization interface (1)
from gurobipy import GRB            # Gurobi optimization interface (2)

Utility functions and output settings used in this notebook are defined in the two cells below.

In [2]:
def printTime(task, tic, toc):
    """Quick function for nicely printing a time to 5 s.f."""
    print(f"{task} in {toc - tic:0.5f} seconds\n")


def printMatrix(matrix, description="ans =", precision=3):
    """Quick function for nicely printing a matrix"""
    print(f"{description}\n", np.round(matrix, precision))


# want to round rather than truncate when printing
np.set_printoptions(threshold=np.inf)

# only show numpy output to five decimal places
np.set_printoptions(formatter={'float_kind':"{:.5f}".format})

## Standard Problem

In the context of genetic selection, we want to maximize selection of genetic merit (a measure of desirable traits) while minimizing risks due to inbreeding. This can be formed mathematically as a multi-objective optimization problem
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w\right),\ \max_w \left(w^{T}\mu\right) \text{ subject to }\ w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},\ l\leq w\leq u
$$
where $w$ is the vector of proportional contributions, $\Sigma$ is a matrix encoding risk, $\mu$ is a vector encoding returns, $l$ encodes lower bounds on contributions, $u$ encodes upper bounds on contributions, $\mathcal{S}$ is an index set of candidates who are sires, and $\mathcal{D}$ is an index set of candidates who are dams.

When using Critical Line Algorithm we typically frame the problem as minimizing risk for a target level of return, $\mu_p$, and can formulate the problem as a single objective
$$
    \min_w \left(\frac{1}{2}w^{T}\Sigma w - \lambda w^{T}\mu\right) \text{ subject to }\ w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},\ l\leq w\leq u.
$$
In this representation of the problem, $\lambda$ is a control variable which balances how we trade of between risk and return. Each value of $\lambda$ will give a different solution on the critical frontier of the problem.

The mathematics of framing it this way works out nicer than maximizing return for a target level of risk, $\sigma_p$. However, [we will find](Scratch/robust-genetics-reframe.ipynb) that this framing actually makes the mathematics _more_ complicated in the context of robust optimization.

For that reason we will instead reframe the problem as 
$$
    \max_w \left(w^{T}\mu - \frac{\lambda}{2}w^{T}\Sigma w\right) \text{ subject to }\ w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},\ l\leq w\leq u.
$$
Note that the $\lambda$ here corresponding to a particular point on the frontier will not have the same value as the $\lambda$ for the same point in the other framing.

### Constraint Formulation

Since it is beneficial to work with problems in a standard form,
$$
    \min_x \frac{1}{2} x^T A x + q^T x\ \text{ subject to }\ Gx\leq h,\ Mx = m,\ l\leq x\leq u,
$$
we will need to do a very slight rearrangement of the problem to incorporate our two sum-to-half constraints within a single equality constraint. We also do not use the $Gx\leq h$ constraint in our problem.

We observe that the two vector constraints
$$
    w_{\mathcal{S}}^{T}e_{\mathcal{S}}^{} = \frac{1}{2},\ w_{\mathcal{D}}^{T}e_{\mathcal{D}}^{} = \frac{1}{2},
$$
are equivalent to the single matrix constraint
$$
    Mw := \begin{bmatrix}
        \mathbb{I}\lbrace 1\in\mathcal{S}\rbrace & \mathbb{I}\lbrace 2\in\mathcal{S}\rbrace & \cdots & \mathbb{I}\lbrace n\in\mathcal{S}\rbrace \\
        \mathbb{I}\lbrace 1\in\mathcal{D}\rbrace & \mathbb{I}\lbrace 2\in\mathcal{D}\rbrace & \cdots & \mathbb{I}\lbrace n\in\mathcal{D}\rbrace \end{bmatrix}w = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},
$$
where $\mathbb{I}\lbrace i\in\mathcal{I}\rbrace$ is an indicator function denoting whether index $i$ is in the set of indices $\mathcal{I}$.

### Toy Example ($n = 3$)

Lets see how this works in an example. We will start by looking how this problem might be solving using a higher level modelling package, Python's [qpsolvers](https://qpsolvers.github.io/qpsolvers/index.html) library. Consider the problem where
$$
    \mu = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
    \Sigma = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 5 & 0 \\ 0 & 0 & 3 \end{bmatrix}, \quad
    \mathcal{S} = \lbrace 1 \rbrace, \quad
    \mathcal{D} = \lbrace 2, 3 \rbrace, \quad
    l = {\bf 0}, \quad
    u = {\bf 1}.
$$
We define these variables in Python using the following code.

In [3]:
# KEY PROBLEM VARIABLES
problem_size = 3
expected_breeding_values = np.array([
    1.0,
    5.0,
    2.0
])
relationship_matrix = np.array([
    [1, 0, 0],
    [0, 5, 0],
    [0, 0, 3]
])
sire_indices = [0]
dam_indices  = [1,2]
lower_bound = np.full((problem_size, 1), 0.0)
upper_bound = np.full((problem_size, 1), 1.0)

We have the additional variables which need setting up so that the problem works in `qpsolvers`. 

In [4]:
# OPTIMIZATION SETUP VARIABLES
lam = 2
# define the M so that column i is [1;0] if i is a sire and [0;1] otherwise 
M = np.zeros((2, problem_size))
M[0, sire_indices] = 1
M[1, dam_indices] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])

Finally, we solve the problem using the modules' `solve_qp` function. This utilises Gurobi via an API, a fact which will be important once we start to consider larger problem sizes. 

In [5]:
# SOLVE THE PROBLEM
def solveGS(lam):
    return solve_qp(
        P = lam*relationship_matrix,
        q = -expected_breeding_values,
        G = None,
        h = None,
        A = M,
        b = m,
        lb = lower_bound,
        ub = upper_bound,
        solver = "gurobi"
    )

print(f"QP solution: w = {solveGS(lam)}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
QP solution: w = [0.50000 0.37500 0.12500]


Excellent, we have code which is telling us that our optimal contributions for $\lambda = 2$ are $w_1 = 0.5$, $w_2 = 0.375$, and $w_3 = 0.125$. We could vary our $\lambda$ value too and find other points on the frontier.¹

In [6]:
for x in range(1,6):
    print(f"lambda = {x}: w = {solveGS(x)}")

lambda = 1: w = [0.50000 0.50000 0.00000]
lambda = 2: w = [0.50000 0.37500 0.12500]
lambda = 3: w = [0.50000 0.31250 0.18750]
lambda = 4: w = [0.50000 0.28125 0.21875]
lambda = 5: w = [0.50000 0.26250 0.23750]


## Robust Optimization

A wrinkle to this is that we don't typically our problem variables with certainty. There _are_ ways to define $\Sigma$ based on relationships within the cohort (which are known and discussed more later) but $\mu$ is an estimated variable. In particular we say $\mu$ has a univariate normal distribution, $\mu\sim N(\bar{\mu}, \Omega)$. This means we must turn to optimization tools which can address this uncertainty.²

Robust optimization is one such tool in which we adjust the objective function to model the inherent uncertainty in the problem. Within this framework, the problem would be the bilevel optimization problem
$$
    \max_w \left(\min_{\mu\in U_{\mu}} \left(w^{T}\mu \right) - \frac{\lambda}{2}w^{T}\Sigma w\right) \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
The result of the inner problem determines the outer problem in a follower-leader setup. However, the inner problem is relatively simple and can be solved explicitly based on our choice of $U_{\mu}$, representing an uncertainty set around the value of $\mu$.


### Quadratic Uncertainty Sets

There are many ways to define the uncertainty set $U_{\mu}$, but for practical reasons relating to continuity and differentiability it's beneficial to define it using a quadratic uncertainty set.³ We say that
$$
    U_{\mu} := \left\lbrace \mu :\ {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \leq \kappa^2 \right\rbrace
$$
which in effect bounds the uncertainty in a ball about $\bar{\mu}$ with parameter $\kappa$.⁴ This means that our lower level problem $\min_{\mu\in U_{\mu}} (w^{T}\mu)$ becomes
$$
    \min_{\mu} \left(w^{T}\mu \right)\quad \text{subject to}\quad {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \leq \kappa^2,
$$
or, in standard form,
$$
    \min_{\mu} \left(w^{T}\mu\right) \quad\text{subject to}\quad \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \geq 0.
$$
This is convex, since $\Omega$ is a positive definite matrix and $w^{T}\mu$ is an affine function. If we define a Lagrangian multiplier $\rho\in\mathbb{R}$, the KKT conditions of this problem are:
\begin{align}
    \nabla_{\mu}L(\mu, \rho) = 0 \quad&\Rightarrow\quad \nabla_{\mu}\left( w^{T}\mu - \rho\big( \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \big) \right) = 0, \\
    c(\mu) \geq 0                \quad&\Rightarrow\quad \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \geq 0, \\
    \rho \geq 0                  \quad&\Rightarrow\quad \rho\geq0, \\
    \rho c(\mu) = 0              \quad&\Rightarrow\quad \rho\left(\kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \right) = 0.
\end{align}
From (1) we have that
$$
    w + \rho 2\Omega^{-1}(\mu-\bar{\mu}) = 0 \quad\Rightarrow\quad \mu - \bar{\mu} = -\frac{1}{2\rho}\Omega w \qquad (5)
$$
\end{align*}
which when substituted into (4) gives
\begin{align*}
    &\phantom{\Rightarrow}\quad \rho\left( \kappa^2 - {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) \right) = 0 \\
    &\Rightarrow\quad \rho\left( \kappa^2 - \big(-\frac{1}{2\rho}\Omega w\big)^{T}\Omega^{-1}\big(-\frac{1}{2\rho}\Omega w\big) \right) = 0 \\
    &\Rightarrow\quad \rho\kappa^2 - \frac{1}{4\rho}w^{T}\Omega^{T}\Omega^{-1}\Omega w = 0 \\
    &\Rightarrow\quad \rho^2\kappa^2 = \frac{1}{4}w^{T}\Omega w\quad \text{(since $\Omega$ symmetric)} \\
    &\Rightarrow\quad \rho\kappa = \frac{1}{2}\sqrt{w^{T}\Omega w}\quad \text{(since $\rho,\kappa\geq0$)} \\
    &\Rightarrow\quad \frac{1}{2\rho} = \frac{\kappa}{\sqrt{w^{T}\Omega w}}.
\end{align*}
Substituting this back into (5), we find that
$$
    \mu - \bar{\mu} = -\frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w \quad\Rightarrow\quad \mu = \bar{\mu} - \frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w
$$
as the solution for the inner problem. Substituting this back into the outer problem gives
$$
    \max_w \left(w^{T}\left( \bar{\mu} - \frac{\kappa}{\sqrt{w^{T}\Omega w}}\Omega w \right) - \frac{\lambda}{2}w^{T}\Sigma w\right) \ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
which after simplifying down and rationalizing the denominator becomes
$$
    \max_w \left(w^{T}\bar{\mu} - \kappa\sqrt{w^{T}\Omega w} - \frac{\lambda}{2}w^{T}\Sigma w\right) \ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
where $\kappa\in\mathbb{R}$ is our robust optimization parameter. Since our objective has gained an additional square root term, this is obviously no longer a quadratic problem and `qpsolvers` is no longer a viable tool. We will instead now need to work with the Gurobi API directly.

### Using Gurobi

To illustrate how the setup changes when uncertainty is added, we will first look at how Gurobi handles the standard problem. The following code returns the same solution as 

In [7]:
# create a model for standard genetic selection
model = gp.Model("standardGS")

# define variable of interest as a continuous 
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")

# set the objective function
model.setObjective(
    w.transpose()@expected_breeding_values - (lam/2)*w@(relationship_matrix@w),
GRB.MAXIMIZE)

# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints
model.addConstr(w >= lower_bound, name="lower-bound")
model.addConstr(w <= upper_bound, name="upper-bound")

# solve the problem with Gurobi
model.optimize()
print(f"w = {w.X}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 3 columns and 24 nonzeros
Model fingerprint: 0xd5877ae8
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     

Unfortunately Gurobi cannot handle our robust problem with its objective function in the form
$$
    \max_w \left(w^{T}\bar{\mu} - \kappa\sqrt{w^{T}\Omega w} - \frac{\lambda}{2}w^{T}\Sigma w\right) \ \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
or in its norm form
$$
    \max_w \left(w^{T}\bar{\mu} - \kappa\|\Omega^{\frac{1}{2}}w\| - \frac{\lambda}{2}w^{T}\Sigma w\right)  \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u,
$$
so some further adjustments are needed first. If we define a real auxillary variable $z\geq0$ such that $z\geq\sqrt{w^{T}\Omega w}$, then our problem becomes
$$
    \max_w \left(w^{T}\bar{\mu} - \kappa z - \frac{\lambda}{2}w^{T}\Sigma w\right) \ \text{ subject to }\ z\geq\sqrt{w^{T}\Omega w},\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$
Since $z>0$, $\kappa\geq0$, and we're maximizing an objective containing "$-\kappa z$", this term of the objective will be biggest when $z$ is smallest. This will happen precisely when it attains its lower bound from the constraint $z\geq\sqrt{w^{T}\Omega w}$, hence our relaxation is justified since $z$ will push downwards.

However, Gurobi _still_ can't handle this due to the presence of the square root, so we further make use of both $z$ and $\sqrt{w^{T}\Omega w}$ being positive to note that $z\geq\sqrt{w^{T}\Omega w}$ can be squared on both sides:
$$
    \max_w \left(w^{T}\bar{\mu} - \kappa z - \frac{\lambda}{2}w^{T}\Sigma w\right) \ \text{ subject to }\ z^2\geq w^{T}\Omega w,\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
$$

To explore this with our toy problem from before, we will define
$$
    \bar{\mu} = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
    \Omega = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 4 & 0 \\ 0 & 0 & \frac{1}{8} \end{bmatrix},\quad
    \kappa = 0.5
$$
and retain the other variables as before.

In [8]:
omega = np.array([
    [1, 0, 0],
    [0, 4, 0],
    [0, 0, 1/8]
])

kappa = 0.5

We then formulate this in Python as follows.

In [9]:
# create a new model for robust genetic selection
model = gp.Model("robustGS")

# define variables of interest as a continuous
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
z = model.addVar(lb=0.0, name="z")

# setup the robust objective function
model.setObjective(
    w.transpose()@expected_breeding_values - (lam/2)*w@(relationship_matrix@w) - kappa*z,
GRB.MAXIMIZE)

# add quadratic uncertainty constraint⁵
model.addConstr(z**2 >= w.transpose()@omega@w, name="uncertainty")
# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints
model.addConstr(w >= lower_bound, name="lower-bound")
model.addConstr(w <= upper_bound, name="upper-bound")

# solve the problem with Gurobi
model.optimize()
print(f"w = {w.X},\nz = {z.X:.5f}.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 4 columns and 24 nonzeros
Model fingerprint: 0xe9e68b83
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-01, 4e+00]
  Objective range  [5e-01, 5e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 8 columns, 11 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.700e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residu

We can repeat this experiment with varying $\kappa$ to see how how different tolerances for uncertainty impact the robust turning point returned.

| $\kappa$ |            $w$            |   $z$   | $f(w,z)$ |
| -------: | ------------------------: | ------: | -------: |
|     0.00 | [0.50000 0.37500 0.12500] | 3.52875 |  1.62499880e+00
|     0.25 | [0.50000 0.34998 0.15002] | 0.86184 |  1.40453330e+00
|     0.50 | [0.50000 0.32623 0.17377] | 0.82431 |  1.19381906e+00
|     0.75 | [0.50000 0.30443 0.19557] | 0.79089 |  9.91998750e-01
|     1.00 | [0.50000 0.28391 0.21609] | 0.75044 |  7.98188968e-01
|     2.00 | [0.50000 0.21877 0.28123] | 0.67181 |  8.61190139e-02
|     4.00 | [0.50000 0.14644 0.35356] | 0.59279 | -1.16409198e+00
|     8.00 | [0.50000 0.09102 0.40898] | 0.55140 | -3.43139120e+00
|    16.00 | [0.50000 0.05658 0.44342] | 0.53608 | -7.76342696e+00
|    32.00 | [0.50000 0.03695 0.46305] | 0.53128 | -1.62903237e+01
|    64.00 | [0.50000 0.02635 0.47365] | 0.52992 | -3.32626440e+01

We can see that in the case $\kappa = 0$ we return the standard optimization solution, as expected. However, we do have $z\neq0$ which suggests that a small amount of numerical error has been introduced as a result of asking Gurobi to optimize $z$ as well as $w$. We also not that as $\kappa\rightarrow\infty$, we tend towards the portfolio of maximum risk.

TODO _do a comparison of the value $z$ against $\sqrt{w\Omega w}$, analyzing the gap between the two. I suspect this will explain away what I originally thought was "numerical error"_

### Verifying with KKTs

As with other(?) optimization problems, we can verify that we have produced an optimum solution by checking that it satisfies the KKT conditions.

TODO: _check that these values are correct, talk about subbing into the KKT conditions._

## Real (Simulated) Data

Now we've looked at how these problems can be approached with Gurobi, we can try an example with realistic simulated data. Here we have data for a cohort of 50 candidates split across three data files, each containing space-separated values.

1. In `A50.txt` the matrix $\Sigma$ is described, with columns for $i$, $j$, and $a_{ij}$, where only the upper triangle is described since the matrix is symmetric.
2. In `EBV50.txt` the vector $\bar{\mu}$ is described, with only one column containing the posterior mean over 1000 samples.
3. In `S50.txt` the matrix $\Omega$ is described, with columns for $i$, $j$, and $\sigma_{ij}$, where only the upper triangle is described since the matrix is symmetric.

Note that this particular problem does not contain a separate file for sex data; odd indexed candidates are sires and even indexed candidates are dams. For now, we also don't have a $l$ or $u$ bounding the possible weights.


The first step then is clearly going to be loading the matrices in particular from file. We create the following function to do so.


In [10]:
def load_problem(A_filename, E_filename, S_filename, dimension=False):
    """
    Used to load genetic selection problems into NumPy. It takes three
    string inputs for filenames where Sigma, Mu, and Omega are stored,
    as well as an optional integer input for problem dimension if this
    is known. If it's know know, it's worked out based on E_filename.

    As output, it returns (A, E, S, n), where A and S are n-by-n NumPy
    arrays, E is a length n NumPy array, and n is an integer.
    """

    def load_symmetric_matrix(filename, dimension):
        """
        Since NumPy doesn't have a stock way to load matrices
        stored in coordinate format format, this adds one.
        """

        matrix = np.zeros([dimension, dimension])

        with open(filename, 'r') as file:
            for line in file:
                i, j, entry = line.split(" ")
                # data files indexed from 1, not 0
                matrix[int(i)-1, int(j)-1] = entry
                matrix[int(j)-1, int(i)-1] = entry

        return matrix


    # if dimension wasn't supplied, need to find that
    if not dimension:
        # get dimension from EBV, since it's the smallest file
        with open(E_filename, 'r') as file:
            dimension = sum(1 for _ in file)

    # EBV isn't in coordinate format so can be loaded directly
    E = np.loadtxt(E_filename)  
    # A and S are stored by coordinates so need special loader
    A = load_symmetric_matrix(A_filename, dimension)
    S = load_symmetric_matrix(S_filename, dimension)

    return A, E, S, dimension

For the given example problem we can now solve it with Gurobi using the exact same methods as before, for both the standard genetic selection problem and the robust version of the problem. The following cell does both alongside each other to accentuate the differences.

In [11]:
sigma, mubar, omega, n = load_problem(
    "Example/A50.txt",
    "Example/EBV50.txt",
    "Example/S50.txt",
    50
)

lam = 1
kappa = 10

# define the M so that column i is [1;0] if i is a sire (so even) and [0;1] otherwise 
M = np.zeros((2, n))
M[0, range(0,50,2)] = 1
M[1, range(1,50,2)] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])

# create models for standard and robust genetic selection
model_std = gp.Model("n50standard")
model_rbs = gp.Model("n50robust")

# initialise w for both models, z for robust model
w_std = model_std.addMVar(shape=n, lb=0.0, vtype=GRB.CONTINUOUS, name="w") 
w_rbs = model_rbs.addMVar(shape=n, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
z_rbs = model_rbs.addVar(lb=0.0, name="z")

# define the objective functions for both models
model_std.setObjective(
    w_std.transpose() @ mubar - (lam/2)*w_std.transpose()@(sigma@w_std),
GRB.MAXIMIZE)

model_rbs.setObjective(
    # Gurobi does offer a way to set one objective in terms of another, i.e.
    # we could use `model_std.getObjective() - lam*kappa*z_rbs` to define this
    # robust objective, but it results in a significant slowdown in code.
    w_rbs.transpose() @ mubar - (lam/2)*w_rbs.transpose()@(sigma@w_rbs) - kappa*z_rbs,
GRB.MAXIMIZE)

# add sum-to-half constraints to both models
model_std.addConstr(M @ w_std == m, name="sum-to-half")
model_rbs.addConstr(M @ w_rbs == m, name="sum-to-half")

# add quadratic uncertainty constraint to the robust model
model_rbs.addConstr(z_rbs**2 <= w_rbs.transpose()@omega@w_rbs, name="uncertainty")

# since working with non-trivial size, set a time limit
time_limit = 60*5  # 5 minutes
model_std.setParam(GRB.Param.TimeLimit, time_limit)
model_std.setParam(GRB.Param.TimeLimit, time_limit)

# for the same reason, also set a duality gap tolerance
duality_gap = 0.05
model_std.setParam('MIPGap', duality_gap)
model_rbs.setParam('MIPGap', duality_gap)

# produce model outputs for external usage
model_std.write("MeanVar.mps")
model_rbs.write("Robust.mps")

# solve both problems with Gurobi
model_std.optimize()
model_rbs.optimize()

# HACK code which prints the results for comparison in a nice format
print("\nSIRE WEIGHTS\t\t\t DAM WEIGHTS")
print("-"*20 + "\t\t " + "-"*20)
print(" i   w_std    w_rbs\t\t  i   w_std    w_rbs")
for candidate in range(25):
    print(f"{candidate*2:02d}  {w_std.X[candidate*2]:.5f}  {w_rbs.X[candidate*2]:.5f} \
            {candidate*2+1:02d}  {w_std.X[candidate*2+1]:.5f}  {w_rbs.X[candidate*2+1]:.5f}")
    
print(f"\nMaximum change: {max(np.abs(w_std.X-w_rbs.X)):.5f}")
print(f"Average change: {np.mean(np.abs(w_std.X-w_rbs.X)):.5f}")
print(f"Minimum change: {min(np.abs(w_std.X-w_rbs.X)):.5f}")

Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.05
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 50 columns and 100 nonzeros
Model fingerprint: 0x32c74476
Model has 1275 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [5e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 5e-01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 50 columns, 50 nonzeros
Presolved model has 1275 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 49
 AA' NZ     : 1.274e+03
 Factor NZ  : 1.326e+03
 Factor Ops : 4.553e+04 (less than 1 second per iteration)
 Threa

We can see that with parameters $\lambda = 1, \kappa = 10$, we go from having most candidates having zero contribution in the standard solution to most candidates having a contribution of order 0.0001 in the robust solution.

BUG: _with issue regarding Gurobi and `np.inner` fixed, there's now little-if-any difference between the standard and robust solutions. Need to look at what's going on here._

## Footnotes

1.
   Working in terms of target risk ($\sigma_p$) rather than target return ($\mu_p$) means we cannot work through the full range of $\lambda$ as easily. The "end point" of the frontier which previously corresponded to $\lambda = 0$ now instead corresponds to $\lambda = \infty$ (although practically, a value of $\lambda = 10^5$ recovers it).

2.
   In reality, $\Omega$ is also an estimated variable but we can ignore this for now to avoid going down a rabbit hole of uncertainties.

3.
   For example, we could alternatively define $U_{\mu}$ using a box uncertainty set
   $$
      U_{\mu} := \left\lbrace \mu :\ | \mu_i - \bar{\mu}_i| \leq\xi_i,\ \forall i \right\rbrace
   $$
   where $\xi$ is some upper bound on the uncertainty value. In this case, work by [Heckel et al (2016)](https://www.risk.net/journal-of-investment-strategies/2475975/insights-into-robust-optimization-decomposing-into-mean-variance-and-risk-based-portfolios) shows that our objective function would gain an additional absolute value term, becoming
   $$
      \max_w \left(w^{T}\mu - \max_i(\xi_i|w_i|) - \frac{\lambda}{2}w^{T}\Sigma w\right)  \text{ subject to }\ Mw = \begin{bmatrix} 0.5 \\ 0.5\end{bmatrix},\ l\leq w\leq u.
   $$
   This absolute value term means we lose many of the properties that we would like to work with in our objective functions.

4.
   By "ball" we do not just mean a sphere in the typical geometric sense, but a ball in the sense of metric spaces. If we recognise that in our definition we have the transpose of a vector, multiplied by a positive definite matrix, multiplied by the same vector, we can see that
   \begin{align*}
      {(\mu-\bar{\mu})}^{T}\Omega^{-1}(\mu-\bar{\mu}) &=: x^{T}Ax \\
                                                      &= x^{T}L^{T}Lx \quad \text{(using Cholesky factorisation)} \\
                                                      &= {(Lx)}^{T}{(Lx)} \\
                                                      &= {\|Lx\|}_2^2 \\
                                                      &= {\|x\|}_{L^{-1}}^2 \leq \kappa^2.  
   \end{align*}
   <!-- TODO I got up to the four line by myself but hadn't come across the last line before Julian suggested it, so need to make sure I'm comfortable with how these special matrix norms work. -->
   In other words, in the norm of $\Omega$'s Cholesky factor our uncertainty set is a ball of radius $\kappa$ centred on $\bar{\mu}$.

5.
   Due to issues with Gurobi, it's important that our uncertainty constraint is modelled as
   ```python3
   model.addConstr(z**2 <= w.transpose()@omega@w, name="uncertainty")
   ```
   rather than something like
   ```python3
   model.addConstr(z**2 <= np.inner(w, omega@w), name="uncertainty")
   ```
   For reasons unknown, Gurobi treats these two constraints differently.